In [1]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import pandas as pd

In [2]:
tweets = pd.read_csv('train.csv') 
tests = pd.read_csv('test.csv') 

In [3]:
tweets.drop_duplicates(subset = 'text', keep = False, inplace = True)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7434 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7434 non-null   int64 
 1   keyword   7378 non-null   object
 2   location  4982 non-null   object
 3   text      7434 non-null   object
 4   target    7434 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 348.5+ KB


In [4]:
t = Tokenizer()
t.fit_on_texts(tweets['text'])
vocab_size = len(t.word_index) + 1
print(vocab_size)

22586


In [5]:
t.word_index

{'t': 1,
 'co': 2,
 'http': 3,
 'the': 4,
 'a': 5,
 'in': 6,
 'to': 7,
 'of': 8,
 'and': 9,
 'i': 10,
 'is': 11,
 'for': 12,
 'on': 13,
 'you': 14,
 'my': 15,
 'that': 16,
 'it': 17,
 'with': 18,
 'at': 19,
 'by': 20,
 'this': 21,
 'https': 22,
 'from': 23,
 'be': 24,
 'are': 25,
 'was': 26,
 'have': 27,
 'like': 28,
 'amp': 29,
 'me': 30,
 'as': 31,
 'up': 32,
 'but': 33,
 'just': 34,
 'so': 35,
 'not': 36,
 'your': 37,
 'out': 38,
 'no': 39,
 'all': 40,
 'will': 41,
 'after': 42,
 'an': 43,
 'has': 44,
 'when': 45,
 'fire': 46,
 "i'm": 47,
 'get': 48,
 'now': 49,
 'we': 50,
 'new': 51,
 'if': 52,
 'more': 53,
 '2': 54,
 'via': 55,
 'about': 56,
 'or': 57,
 'news': 58,
 'what': 59,
 'they': 60,
 'one': 61,
 'how': 62,
 'people': 63,
 'he': 64,
 "it's": 65,
 "don't": 66,
 'been': 67,
 'who': 68,
 'over': 69,
 'into': 70,
 'do': 71,
 'video': 72,
 'can': 73,
 'emergency': 74,
 "'": 75,
 'there': 76,
 'disaster': 77,
 'police': 78,
 'than': 79,
 '3': 80,
 'u': 81,
 'her': 82,
 'his': 83,

In [27]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(tweets['text'])
enconded_test = t.texts_to_sequences(tests['text'])

In [28]:
from keras.preprocessing.sequence import pad_sequences

max_length = 31 # Maxima cantidad de palabras en los tweets
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_tests = pad_sequences(enconded_test, maxlen=max_length, padding='post')
print(padded_docs)

[[ 116 4534   25 ...    0    0    0]
 [ 182   46  242 ...    0    0    0]
 [  40 1705 1572 ...    0    0    0]
 ...
 [ 106  225  453 ...    0    0    0]
 [ 121  837 1338 ...    0    0    0]
 [   4  201   53 ...    0    0    0]]


In [8]:
import numpy as np

embeddings_index = dict()
f = open('glove.twitter.27B.100d.txt') # Vectores entrenados de 100 dimensiones
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [9]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # Si la palabra no esta queda llena de 0s
        embedding_matrix[i] = embedding_vector

In [20]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

model1 = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=False)
model1.add(e)
model1.add(layers.Conv1D(256, 7, activation='relu'))
model1.add(Flatten())
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 31, 100)           2258600   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 25, 256)           179456    
_________________________________________________________________
flatten_3 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6401      
Total params: 2,444,457
Trainable params: 185,857
Non-trainable params: 2,258,600
_________________________________________________________________
None


In [21]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = \
train_test_split(padded_docs, tweets['target'], test_size = 0.25, random_state = 123)

In [22]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

model1.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=50,
          verbose=2,
          callbacks=callback)

Epoch 1/50
44/44 - 1s - loss: 0.5262 - accuracy: 0.7444 - val_loss: 0.4449 - val_accuracy: 0.7983
Epoch 2/50
44/44 - 1s - loss: 0.3909 - accuracy: 0.8335 - val_loss: 0.4308 - val_accuracy: 0.8133
Epoch 3/50
44/44 - 1s - loss: 0.3256 - accuracy: 0.8687 - val_loss: 0.4464 - val_accuracy: 0.8144
Epoch 00003: early stopping


In [25]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

model1 = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=False)
model1.add(e)
model1.add(layers.Conv1D(256, 7, activation='relu'))
model1.add(Flatten())
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model1.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 31, 100)           2258600   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 25, 256)           179456    
_________________________________________________________________
flatten_5 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6401      
Total params: 2,444,457
Trainable params: 185,857
Non-trainable params: 2,258,600
_________________________________________________________________
None


In [26]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

model1.fit(padded_docs, tweets['target'],
          batch_size=128,
          epochs=5,
          verbose=2,
          callbacks=callback)

Epoch 1/5
59/59 - 1s - loss: 0.5096 - accuracy: 0.7612
Epoch 2/5
59/59 - 1s - loss: 0.3848 - accuracy: 0.8366
Epoch 3/5
59/59 - 1s - loss: 0.3255 - accuracy: 0.8680
Epoch 4/5
59/59 - 1s - loss: 0.2696 - accuracy: 0.8932
Epoch 5/5
59/59 - 1s - loss: 0.2180 - accuracy: 0.9257


In [29]:
test_result = model1.predict(padded_tests)

In [32]:
test_result

array([[0.4535236 ],
       [0.6374486 ],
       [0.51281166],
       ...,
       [0.7867126 ],
       [0.6981692 ],
       [0.24020772]], dtype=float32)

In [40]:
submit = []

for i in test_result:
    if i >= 0.5 :
        submit.append(1)
    else:
        submit.append(0)

submit

[0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,


In [49]:
tests

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,1
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,1
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,1


In [51]:
submit_df = tests[['id', 'target']]

In [52]:
submit_df

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [53]:
submit_df.to_csv('submit_prueba_36.csv', index=False)

In [54]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

model1 = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=False)
model1.add(e)
model1.add(layers.Conv1D(256, 7, activation='relu'))
model1.add(Flatten())
model1.add(Dense(10, activation='sigmoid'))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model1.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 31, 100)           2258600   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 25, 256)           179456    
_________________________________________________________________
flatten_6 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                64010     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 2,502,077
Trainable params: 243,477
Non-trainable params: 2,258,600
_________________________________________________________________
None


In [55]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

model1.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=50,
          verbose=2,
          callbacks=callback)

Epoch 1/50
44/44 - 1s - loss: 0.5305 - accuracy: 0.7365 - val_loss: 0.4629 - val_accuracy: 0.8010
Epoch 2/50
44/44 - 1s - loss: 0.4246 - accuracy: 0.8240 - val_loss: 0.4222 - val_accuracy: 0.8144
Epoch 3/50
44/44 - 1s - loss: 0.3736 - accuracy: 0.8560 - val_loss: 0.4243 - val_accuracy: 0.8112
Epoch 00003: early stopping


In [76]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

def build_model(fully_conected, num_filters, kern_size):
    model1 = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
    model1.add(e)
    model1.add(layers.Conv1D(num_filters, kern_size, activation='relu'))
    model1.add(Flatten())
    model1.add(Dense(fully_conected, activation='sigmoid'))
    model1.add(Dense(1, activation='sigmoid'))

    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model1

In [77]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[32, 128, 144, 256],
                      kern_size=[3, 5, 7],
                      batch_size = [45,65,76,88],
                      fully_conected = [30, 60, 800], epochs = [15])

model = KerasClassifier(build_fn=build_model, epochs=15, validation_split=0.1,verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1,scoring = 'accuracy')

grid_result = grid.fit(x_train, y_train, callbacks=[callback])


test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] num_filters=128, kern_size=7, fully_conected=800, epochs=15, batch_size=88 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


43/43 [==============================] - 1s 35ms/step - loss: 0.6060 - accuracy: 0.7068 - val_loss: 0.4906 - val_accuracy: 0.7661
Epoch 2/15
43/43 [==============================] - 1s 32ms/step - loss: 0.3714 - accuracy: 0.8437 - val_loss: 0.4632 - val_accuracy: 0.7804
Epoch 3/15
43/43 [==============================] - 1s 32ms/step - loss: 0.2524 - accuracy: 0.9003 - val_loss: 0.5939 - val_accuracy: 0.7637
Epoch 00003: early stopping
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
16/16 [==============================] - 0s 5ms/step
[CV]  num_filters=128, kern_size=7, fully_conected=800, epochs=15, batch_size=88, total=   5.1s
[CV] num_filters=128, kern_size=7, fully_conected=800, epochs=15, batch_size=

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s remaining:    0.0s


43/43 [==============================] - 1s 34ms/step - loss: 0.5289 - accuracy: 0.7443 - val_loss: 0.4674 - val_accuracy: 0.7852
Epoch 2/15
43/43 [==============================] - 1s 33ms/step - loss: 0.3331 - accuracy: 0.8623 - val_loss: 0.5105 - val_accuracy: 0.7804
Epoch 00002: early stopping
16/16 [==============================] - 0s 7ms/step
[CV]  num_filters=128, kern_size=7, fully_conected=800, epochs=15, batch_size=88, total=   3.7s
[CV] num_filters=128, kern_size=7, fully_conected=800, epochs=15, batch_size=88 
Epoch 1/15
43/43 [==============================] - 2s 35ms/step - loss: 0.5728 - accuracy: 0.7275 - val_loss: 0.4700 - val_accuracy: 0.7828
Epoch 2/15
43/43 [==============================] - 1s 32ms/step - loss: 0.3554 - accuracy: 0.8472 - val_loss: 0.4664 - val_accuracy: 0.7852
Epoch 3/15
43/43 [==============================] - 1s 34ms/step - loss: 0.2232 - accuracy: 0.9133 - val_loss: 0.4796 - val_accuracy: 0.7804
Epoch 00003: early stopping
16/16 [=============

19/19 [==============================] - 0s 2ms/step
[CV]  num_filters=128, kern_size=3, fully_conected=30, epochs=15, batch_size=76, total=   6.7s
[CV] num_filters=128, kern_size=3, fully_conected=30, epochs=15, batch_size=76 
Epoch 1/15
50/50 [==============================] - 1s 23ms/step - loss: 0.5407 - accuracy: 0.7305 - val_loss: 0.5184 - val_accuracy: 0.7613
Epoch 2/15
50/50 [==============================] - 1s 24ms/step - loss: 0.3922 - accuracy: 0.8328 - val_loss: 0.4679 - val_accuracy: 0.7780
Epoch 3/15
50/50 [==============================] - 1s 26ms/step - loss: 0.3147 - accuracy: 0.8753 - val_loss: 0.4599 - val_accuracy: 0.7924
Epoch 4/15
50/50 [==============================] - 1s 23ms/step - loss: 0.2365 - accuracy: 0.9213 - val_loss: 0.4797 - val_accuracy: 0.8043
Epoch 00004: early stopping
19/19 [==============================] - 0s 2ms/step
[CV]  num_filters=128, kern_size=3, fully_conected=30, epochs=15, batch_size=76, total=   5.6s
[CV] num_filters=128, kern_size=

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.5min finished


67/67 [==============================] - 1s 21ms/step - loss: 0.5192 - accuracy: 0.7403 - val_loss: 0.4657 - val_accuracy: 0.7867
Epoch 2/15
67/67 [==============================] - 1s 21ms/step - loss: 0.3811 - accuracy: 0.8380 - val_loss: 0.4836 - val_accuracy: 0.7849
Epoch 00002: early stopping
25/25 [==============================] - 0s 2ms/step
Best Accuracy : 0.8083
{'num_filters': 128, 'kern_size': 3, 'fully_conected': 60, 'epochs': 15, 'batch_size': 76}





In [79]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
model.add(e)
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(60, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [80]:
x_train, x_test, y_train, y_test = \
train_test_split(padded_docs, tweets['target'], test_size = 0.25, random_state = 123)
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=76,
          epochs=15,
          verbose=2,
          callbacks=callback)

Epoch 1/15
74/74 - 1s - loss: 0.5077 - accuracy: 0.7516 - val_loss: 0.4315 - val_accuracy: 0.8112
Epoch 2/15
74/74 - 1s - loss: 0.3732 - accuracy: 0.8348 - val_loss: 0.4110 - val_accuracy: 0.8289
Epoch 3/15
74/74 - 1s - loss: 0.2807 - accuracy: 0.8874 - val_loss: 0.4254 - val_accuracy: 0.8219
Epoch 00003: early stopping


In [93]:
model2 = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
model2.add(e)
model2.add(layers.Conv1D(128, 3, activation='relu'))
model2.add(Flatten())
model2.add(Dense(60, activation='sigmoid'))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [94]:
model2.fit(padded_docs, tweets['target'],
          batch_size=76,
          epochs=2,
          verbose=2)

Epoch 1/2
98/98 - 2s - loss: 0.5198 - accuracy: 0.7441
Epoch 2/2
98/98 - 2s - loss: 0.3652 - accuracy: 0.8445


In [95]:
test_result = model1.predict(padded_tests)

In [96]:
test_result

array([[0.674353  ],
       [0.80772024],
       [0.6589092 ],
       ...,
       [0.8821541 ],
       [0.8534104 ],
       [0.3019307 ]], dtype=float32)

In [97]:
submit = []

for i in test_result:
    if i >= 0.5 :
        submit.append(1)
    else:
        submit.append(0)

submit

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [98]:
tests['target'] = submit

In [99]:
submit_df = tests[['id', 'target']]

In [100]:
submit_df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [89]:
submit_df.to_csv('submit_prueba_37.csv', index=False)

In [106]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

def build_model(fully_conected, num_filters, kern_size):
    model1 = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
    model1.add(e)
    model1.add(layers.Conv1D(128, 3, activation='relu'))
    model1.add(layers.Conv1D(num_filters, kern_size, activation='relu'))
    model1.add(Flatten())
    model1.add(Dense(fully_conected, activation='sigmoid'))
    model1.add(Dense(1, activation='sigmoid'))

    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model1

In [108]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[ 144, 256],
                      kern_size=[3, 5, 7],
                      batch_size = [76,88],
                      fully_conected = [60, 800], epochs = [15])

model = KerasClassifier(build_fn=build_model, epochs=15, validation_split=0.1,verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1,scoring = 'accuracy')

grid_result = grid.fit(x_train, y_train, callbacks=[callback])


test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] num_filters=256, kern_size=3, fully_conected=60, epochs=15, batch_size=76 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


50/50 [==============================] - 1s 29ms/step - loss: 0.5217 - accuracy: 0.7584 - val_loss: 0.4831 - val_accuracy: 0.7757
Epoch 2/15
50/50 [==============================] - 1s 26ms/step - loss: 0.3787 - accuracy: 0.8394 - val_loss: 0.4705 - val_accuracy: 0.7757
Epoch 3/15
50/50 [==============================] - 1s 26ms/step - loss: 0.2869 - accuracy: 0.8844 - val_loss: 0.4811 - val_accuracy: 0.7924
Epoch 00003: early stopping
19/19 [==============================] - 0s 5ms/step
[CV]  num_filters=256, kern_size=3, fully_conected=60, epochs=15, batch_size=76, total=   4.9s
[CV] num_filters=256, kern_size=3, fully_conected=60, epochs=15, batch_size=76 
Epoch 1/15


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s remaining:    0.0s


50/50 [==============================] - 1s 29ms/step - loss: 0.5228 - accuracy: 0.7414 - val_loss: 0.4945 - val_accuracy: 0.7733
Epoch 2/15
50/50 [==============================] - 1s 27ms/step - loss: 0.3801 - accuracy: 0.8357 - val_loss: 0.4576 - val_accuracy: 0.7828
Epoch 3/15
50/50 [==============================] - 1s 27ms/step - loss: 0.2757 - accuracy: 0.8966 - val_loss: 0.4897 - val_accuracy: 0.7852
Epoch 00003: early stopping
19/19 [==============================] - 0s 5ms/step
[CV]  num_filters=256, kern_size=3, fully_conected=60, epochs=15, batch_size=76, total=   5.0s
[CV] num_filters=256, kern_size=3, fully_conected=60, epochs=15, batch_size=76 
Epoch 1/15
50/50 [==============================] - 1s 29ms/step - loss: 0.5473 - accuracy: 0.7289 - val_loss: 0.4920 - val_accuracy: 0.7828
Epoch 2/15
50/50 [==============================] - 1s 27ms/step - loss: 0.3826 - accuracy: 0.8315 - val_loss: 0.4769 - val_accuracy: 0.7804
Epoch 3/15
50/50 [==============================] 

50/50 [==============================] - 2s 37ms/step - loss: 0.5484 - accuracy: 0.7259 - val_loss: 0.5159 - val_accuracy: 0.7613
Epoch 2/15
50/50 [==============================] - 2s 38ms/step - loss: 0.3952 - accuracy: 0.8251 - val_loss: 0.4746 - val_accuracy: 0.7733
Epoch 3/15
50/50 [==============================] - 2s 32ms/step - loss: 0.3010 - accuracy: 0.8759 - val_loss: 0.4566 - val_accuracy: 0.7900
Epoch 4/15
50/50 [==============================] - 2s 32ms/step - loss: 0.2074 - accuracy: 0.9224 - val_loss: 0.5796 - val_accuracy: 0.7661
Epoch 00004: early stopping
19/19 [==============================] - 0s 4ms/step
[CV]  num_filters=144, kern_size=7, fully_conected=60, epochs=15, batch_size=76, total=   7.9s
[CV] num_filters=144, kern_size=7, fully_conected=60, epochs=15, batch_size=76 
Epoch 1/15
50/50 [==============================] - 2s 34ms/step - loss: 0.5541 - accuracy: 0.7166 - val_loss: 0.4849 - val_accuracy: 0.7780
Epoch 2/15
50/50 [==============================] 

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  2.5min finished


67/67 [==============================] - 4s 66ms/step - loss: 0.5179 - accuracy: 0.7491 - val_loss: 0.4539 - val_accuracy: 0.7939
Epoch 2/15
67/67 [==============================] - 5s 68ms/step - loss: 0.3825 - accuracy: 0.8342 - val_loss: 0.4434 - val_accuracy: 0.7921
Epoch 3/15
67/67 [==============================] - 4s 66ms/step - loss: 0.3046 - accuracy: 0.8728 - val_loss: 0.4469 - val_accuracy: 0.8047
Epoch 00003: early stopping
25/25 [==============================] - 0s 10ms/step
Best Accuracy : 0.8074
{'num_filters': 256, 'kern_size': 5, 'fully_conected': 800, 'epochs': 15, 'batch_size': 76}





In [ ]:
para submitear

In [109]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

def build_model(fully_conected, num_filters, kern_size):
    model1 = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
    model1.add(e)
    model1.add(layers.Conv1D(num_filters, kern_size, activation='relu'))
    model1.add(layers.Conv1D(128, 3, activation='relu'))
    model1.add(Flatten())
    model1.add(Dense(fully_conected, activation='sigmoid'))
    model1.add(Dense(1, activation='sigmoid'))

    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model1

In [110]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[144, 256],
                      kern_size=[3, 5, 7],
                      batch_size = [76,88],
                      fully_conected = [60, 800], epochs = [15])

model = KerasClassifier(build_fn=build_model, epochs=15, validation_split=0.1,verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1,scoring = 'accuracy')

grid_result = grid.fit(x_train, y_train, callbacks=[callback])


test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] num_filters=144, kern_size=3, fully_conected=60, epochs=15, batch_size=88 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


43/43 [==============================] - 1s 32ms/step - loss: 0.5438 - accuracy: 0.7228 - val_loss: 0.4972 - val_accuracy: 0.7637
Epoch 2/15
43/43 [==============================] - 1s 27ms/step - loss: 0.4022 - accuracy: 0.8307 - val_loss: 0.4736 - val_accuracy: 0.7804
Epoch 3/15
43/43 [==============================] - 1s 26ms/step - loss: 0.3255 - accuracy: 0.8655 - val_loss: 0.4943 - val_accuracy: 0.7804
Epoch 00003: early stopping
16/16 [==============================] - 0s 3ms/step
[CV]  num_filters=144, kern_size=3, fully_conected=60, epochs=15, batch_size=88, total=   4.5s
[CV] num_filters=144, kern_size=3, fully_conected=60, epochs=15, batch_size=88 
Epoch 1/15


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s


43/43 [==============================] - 1s 30ms/step - loss: 0.5856 - accuracy: 0.6853 - val_loss: 0.5365 - val_accuracy: 0.7470
Epoch 2/15
43/43 [==============================] - 1s 27ms/step - loss: 0.4085 - accuracy: 0.8267 - val_loss: 0.4537 - val_accuracy: 0.7900
Epoch 3/15
43/43 [==============================] - 1s 27ms/step - loss: 0.3208 - accuracy: 0.8737 - val_loss: 0.4892 - val_accuracy: 0.7924
Epoch 00003: early stopping
16/16 [==============================] - 0s 5ms/step
[CV]  num_filters=144, kern_size=3, fully_conected=60, epochs=15, batch_size=88, total=   4.4s
[CV] num_filters=144, kern_size=3, fully_conected=60, epochs=15, batch_size=88 
Epoch 1/15
43/43 [==============================] - 1s 29ms/step - loss: 0.5471 - accuracy: 0.7246 - val_loss: 0.5032 - val_accuracy: 0.7637
Epoch 2/15
43/43 [==============================] - 1s 27ms/step - loss: 0.4042 - accuracy: 0.8206 - val_loss: 0.4665 - val_accuracy: 0.7900
Epoch 3/15
43/43 [==============================] 

50/50 [==============================] - 2s 41ms/step - loss: 0.5332 - accuracy: 0.7363 - val_loss: 0.4817 - val_accuracy: 0.7804
Epoch 2/15
50/50 [==============================] - 2s 40ms/step - loss: 0.3841 - accuracy: 0.8392 - val_loss: 0.4505 - val_accuracy: 0.8091
Epoch 3/15
50/50 [==============================] - 2s 39ms/step - loss: 0.2698 - accuracy: 0.8945 - val_loss: 0.4940 - val_accuracy: 0.7804
Epoch 00003: early stopping
19/19 [==============================] - 0s 8ms/step
[CV]  num_filters=256, kern_size=7, fully_conected=60, epochs=15, batch_size=76, total=   7.0s
[CV] num_filters=256, kern_size=7, fully_conected=60, epochs=15, batch_size=76 
Epoch 1/15
50/50 [==============================] - 2s 40ms/step - loss: 0.5292 - accuracy: 0.7334 - val_loss: 0.4786 - val_accuracy: 0.7709
Epoch 2/15
50/50 [==============================] - 2s 38ms/step - loss: 0.3746 - accuracy: 0.8389 - val_loss: 0.4527 - val_accuracy: 0.7780
Epoch 3/15
50/50 [==============================] 

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.9min finished


58/58 [==============================] - 2s 31ms/step - loss: 0.5197 - accuracy: 0.7530 - val_loss: 0.5530 - val_accuracy: 0.7437
Epoch 2/15
58/58 [==============================] - 2s 29ms/step - loss: 0.3949 - accuracy: 0.8286 - val_loss: 0.4497 - val_accuracy: 0.7993
Epoch 3/15
58/58 [==============================] - 2s 30ms/step - loss: 0.3016 - accuracy: 0.8780 - val_loss: 0.4524 - val_accuracy: 0.8047
Epoch 00003: early stopping
22/22 [==============================] - 0s 4ms/step
Best Accuracy : 0.8052
{'num_filters': 144, 'kern_size': 3, 'fully_conected': 60, 'epochs': 15, 'batch_size': 88}





submit

In [114]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

def build_model(fully_conected, num_filters, kern_size):
    model1 = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
    model1.add(e)
    model1.add(layers.Conv1D(num_filters, kern_size, activation='relu'))
    model1.add(Flatten())
    model1.add(Dense(1, activation='sigmoid'))

    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model1

In [115]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[128, 256],
                      kern_size=[3, 5, 7],
                      batch_size = [76,88],
                      fully_conected = [16, 60], epochs = [15])

model = KerasClassifier(build_fn=build_model, epochs=15, validation_split=0.1,verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1,scoring = 'accuracy')

grid_result = grid.fit(x_train, y_train, callbacks=[callback])


test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] num_filters=256, kern_size=3, fully_conected=16, epochs=15, batch_size=76 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


50/50 [==============================] - 2s 38ms/step - loss: 0.5504 - accuracy: 0.7153 - val_loss: 0.5115 - val_accuracy: 0.7661
Epoch 2/15
50/50 [==============================] - 1s 19ms/step - loss: 0.3740 - accuracy: 0.8379 - val_loss: 0.5020 - val_accuracy: 0.7709
Epoch 3/15
50/50 [==============================] - 1s 19ms/step - loss: 0.2845 - accuracy: 0.8910 - val_loss: 0.4827 - val_accuracy: 0.7661
Epoch 4/15
50/50 [==============================] - 1s 18ms/step - loss: 0.1999 - accuracy: 0.9296 - val_loss: 0.5392 - val_accuracy: 0.7757
Epoch 00004: early stopping
19/19 [==============================] - 0s 3ms/step
[CV]  num_filters=256, kern_size=3, fully_conected=16, epochs=15, batch_size=76, total=   5.4s
[CV] num_filters=256, kern_size=3, fully_conected=16, epochs=15, batch_size=76 
Epoch 1/15


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.4s remaining:    0.0s


50/50 [==============================] - 1s 20ms/step - loss: 0.5394 - accuracy: 0.7299 - val_loss: 0.5007 - val_accuracy: 0.7613
Epoch 2/15
50/50 [==============================] - 1s 19ms/step - loss: 0.3678 - accuracy: 0.8458 - val_loss: 0.4799 - val_accuracy: 0.7828
Epoch 3/15
50/50 [==============================] - 1s 18ms/step - loss: 0.2846 - accuracy: 0.8881 - val_loss: 0.4757 - val_accuracy: 0.7947
Epoch 4/15
50/50 [==============================] - 1s 18ms/step - loss: 0.1982 - accuracy: 0.9296 - val_loss: 0.4925 - val_accuracy: 0.7900
Epoch 00004: early stopping
19/19 [==============================] - 0s 3ms/step
[CV]  num_filters=256, kern_size=3, fully_conected=16, epochs=15, batch_size=76, total=   4.5s
[CV] num_filters=256, kern_size=3, fully_conected=16, epochs=15, batch_size=76 
Epoch 1/15
50/50 [==============================] - 1s 20ms/step - loss: 0.5325 - accuracy: 0.7323 - val_loss: 0.5122 - val_accuracy: 0.7566
Epoch 2/15
50/50 [==============================] 

43/43 [==============================] - 1s 25ms/step - loss: 0.2003 - accuracy: 0.9333 - val_loss: 0.5076 - val_accuracy: 0.7876
Epoch 00004: early stopping
16/16 [==============================] - 0s 3ms/step
[CV]  num_filters=256, kern_size=3, fully_conected=60, epochs=15, batch_size=88, total=   4.9s
[CV] num_filters=256, kern_size=3, fully_conected=60, epochs=15, batch_size=88 
Epoch 1/15
43/43 [==============================] - 1s 24ms/step - loss: 0.5506 - accuracy: 0.7217 - val_loss: 0.5116 - val_accuracy: 0.7828
Epoch 2/15
43/43 [==============================] - 1s 22ms/step - loss: 0.3739 - accuracy: 0.8448 - val_loss: 0.4989 - val_accuracy: 0.7828
Epoch 3/15
43/43 [==============================] - 1s 22ms/step - loss: 0.2827 - accuracy: 0.8873 - val_loss: 0.4705 - val_accuracy: 0.7924
Epoch 4/15
43/43 [==============================] - 1s 23ms/step - loss: 0.2040 - accuracy: 0.9285 - val_loss: 0.5013 - val_accuracy: 0.7804
Epoch 00004: early stopping
16/16 [===============

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.3min finished


58/58 [==============================] - 1s 24ms/step - loss: 0.5176 - accuracy: 0.7459 - val_loss: 0.4745 - val_accuracy: 0.7939
Epoch 2/15
58/58 [==============================] - 1s 22ms/step - loss: 0.3697 - accuracy: 0.8399 - val_loss: 0.4713 - val_accuracy: 0.7957
Epoch 3/15
58/58 [==============================] - 1s 25ms/step - loss: 0.2749 - accuracy: 0.8902 - val_loss: 0.4740 - val_accuracy: 0.7975
Epoch 00003: early stopping
22/22 [==============================] - 0s 3ms/step
Best Accuracy : 0.8031
{'num_filters': 256, 'kern_size': 3, 'fully_conected': 60, 'epochs': 15, 'batch_size': 88}





In [121]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers
from keras import activations

from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

def build_model(fully_conected, num_filters, kern_size):
    model1 = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
    model1.add(e)
    model1.add(layers.Conv1D(num_filters, kern_size, activation='relu'))
    model1.add(layers.Activation(activations.relu))
    model1.add(Flatten())
    model1.add(Dense(1, activation='sigmoid'))

    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model1

In [124]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(num_filters=[64, 128, 256],
                      kern_size=[3, 5, 7],
                      batch_size = [76,88],
                      fully_conected = [16, 60], epochs = [15])

model = KerasClassifier(build_fn=build_model, epochs=15, validation_split=0.1,verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=10, verbose=2, n_iter=5, n_jobs=1,scoring = 'accuracy')

grid_result = grid.fit(x_train, y_train, callbacks=[callback])


test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)

Fitting 10 folds for each of 5 candidates, totalling 50 fits
[CV] num_filters=256, kern_size=5, fully_conected=16, epochs=15, batch_size=88 
Epoch 1/15


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


52/52 [==============================] - 1s 26ms/step - loss: 0.5156 - accuracy: 0.7513 - val_loss: 0.4814 - val_accuracy: 0.7769
Epoch 2/15
52/52 [==============================] - 1s 23ms/step - loss: 0.3523 - accuracy: 0.8523 - val_loss: 0.4989 - val_accuracy: 0.7829
Epoch 00002: early stopping
7/7 [==============================] - 0s 4ms/step
[CV]  num_filters=256, kern_size=5, fully_conected=16, epochs=15, batch_size=88, total=   4.2s
[CV] num_filters=256, kern_size=5, fully_conected=16, epochs=15, batch_size=88 
Epoch 1/15


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


52/52 [==============================] - 1s 24ms/step - loss: 0.5135 - accuracy: 0.7473 - val_loss: 0.4839 - val_accuracy: 0.7470
Epoch 2/15
52/52 [==============================] - 1s 24ms/step - loss: 0.3518 - accuracy: 0.8487 - val_loss: 0.4918 - val_accuracy: 0.7789
Epoch 00002: early stopping
7/7 [==============================] - 0s 4ms/step
[CV]  num_filters=256, kern_size=5, fully_conected=16, epochs=15, batch_size=88, total=   3.2s
[CV] num_filters=256, kern_size=5, fully_conected=16, epochs=15, batch_size=88 
Epoch 1/15
52/52 [==============================] - 1s 25ms/step - loss: 0.5273 - accuracy: 0.7415 - val_loss: 0.4999 - val_accuracy: 0.7530
Epoch 2/15
52/52 [==============================] - 1s 23ms/step - loss: 0.3563 - accuracy: 0.8470 - val_loss: 0.4809 - val_accuracy: 0.7809
Epoch 3/15
52/52 [==============================] - 1s 23ms/step - loss: 0.2522 - accuracy: 0.9032 - val_loss: 0.4997 - val_accuracy: 0.7948
Epoch 00003: early stopping
7/7 [===================

...

Epoch 1/15
60/60 [==============================] - 2s 27ms/step - loss: 0.5147 - accuracy: 0.7533 - val_loss: 0.4786 - val_accuracy: 0.7709
Epoch 2/15
60/60 [==============================] - 2s 26ms/step - loss: 0.3409 - accuracy: 0.8572 - val_loss: 0.4818 - val_accuracy: 0.7948
Epoch 00002: early stopping
8/8 [==============================] - 0s 4ms/step
[CV]  num_filters=256, kern_size=5, fully_conected=16, epochs=15, batch_size=76, total=   3.9s
[CV] num_filters=256, kern_size=5, fully_conected=16, epochs=15, batch_size=76 
Epoch 1/15
60/60 [==============================] - 2s 27ms/step - loss: 0.4966 - accuracy: 0.7637 - val_loss: 0.5117 - val_accuracy: 0.7689
Epoch 2/15
60/60 [==============================] - 2s 31ms/step - loss: 0.3373 - accuracy: 0.8543 - val_loss: 0.4782 - val_accuracy: 0.7769
Epoch 3/15
60/60 [==============================] - 2s 29ms/step - loss: 0.2383 - accuracy: 0.9068 - val_loss: 0.5177 - val_accuracy: 0.7789
Epoch 00003: early stopping
8/8 [========

52/52 [==============================] - 1s 21ms/step - loss: 0.3025 - accuracy: 0.8753 - val_loss: 0.4690 - val_accuracy: 0.7869
Epoch 4/15
52/52 [==============================] - 1s 20ms/step - loss: 0.2281 - accuracy: 0.9147 - val_loss: 0.4881 - val_accuracy: 0.7988
Epoch 00004: early stopping
7/7 [==============================] - 0s 2ms/step
[CV]  num_filters=128, kern_size=3, fully_conected=60, epochs=15, batch_size=88, total=   5.4s
[CV] num_filters=128, kern_size=3, fully_conected=60, epochs=15, batch_size=88 
Epoch 1/15
52/52 [==============================] - 1s 22ms/step - loss: 0.5555 - accuracy: 0.7139 - val_loss: 0.5233 - val_accuracy: 0.7510
Epoch 2/15
52/52 [==============================] - 1s 19ms/step - loss: 0.3967 - accuracy: 0.8224 - val_loss: 0.4832 - val_accuracy: 0.7769
Epoch 3/15
52/52 [==============================] - 1s 19ms/step - loss: 0.3162 - accuracy: 0.8709 - val_loss: 0.4809 - val_accuracy: 0.7849
Epoch 4/15
52/52 [==============================] - 

7/7 [==============================] - 0s 2ms/step
[CV]  num_filters=64, kern_size=5, fully_conected=16, epochs=15, batch_size=88, total=   3.6s
[CV] num_filters=64, kern_size=5, fully_conected=16, epochs=15, batch_size=88 
Epoch 1/15
52/52 [==============================] - 1s 20ms/step - loss: 0.5530 - accuracy: 0.7219 - val_loss: 0.5329 - val_accuracy: 0.7530
Epoch 2/15
52/52 [==============================] - 1s 18ms/step - loss: 0.3945 - accuracy: 0.8271 - val_loss: 0.4955 - val_accuracy: 0.7649
Epoch 3/15
52/52 [==============================] - 1s 19ms/step - loss: 0.3136 - accuracy: 0.8740 - val_loss: 0.4918 - val_accuracy: 0.7769
Epoch 4/15
52/52 [==============================] - 1s 19ms/step - loss: 0.2387 - accuracy: 0.9105 - val_loss: 0.5110 - val_accuracy: 0.7869
Epoch 00004: early stopping
7/7 [==============================] - 0s 3ms/step
[CV]  num_filters=64, kern_size=5, fully_conected=16, epochs=15, batch_size=88, total=   4.7s
[CV] num_filters=64, kern_size=5, fully

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  4.1min finished


67/67 [==============================] - 1s 20ms/step - loss: 0.5288 - accuracy: 0.7421 - val_loss: 0.5232 - val_accuracy: 0.7760
Epoch 2/15
67/67 [==============================] - 1s 19ms/step - loss: 0.3781 - accuracy: 0.8312 - val_loss: 0.4720 - val_accuracy: 0.7975
Epoch 3/15
67/67 [==============================] - 1s 19ms/step - loss: 0.2957 - accuracy: 0.8764 - val_loss: 0.4823 - val_accuracy: 0.8029
Epoch 00003: early stopping
25/25 [==============================] - 0s 2ms/step
Best Accuracy : 0.8045
{'num_filters': 128, 'kern_size': 3, 'fully_conected': 16, 'epochs': 15, 'batch_size': 76}





In [128]:
model1 = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=31, trainable=True)
model1.add(e)
model1.add(layers.Conv1D(256, 5, activation='relu'))
model1.add(layers.Activation(activations.relu))
model1.add(Flatten())
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [126]:
x_train, x_test, y_train, y_test = \
train_test_split(padded_docs, tweets['target'], test_size = 0.25, random_state = 123)
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)
callbacks = [callback]

model1.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=76,
          epochs=2,
          verbose=2,
          callbacks=callback)

Epoch 1/15
74/74 - 2s - loss: 0.5144 - accuracy: 0.7476 - val_loss: 0.4274 - val_accuracy: 0.8101
Epoch 2/15
74/74 - 2s - loss: 0.3513 - accuracy: 0.8502 - val_loss: 0.4149 - val_accuracy: 0.8219
Epoch 3/15
74/74 - 2s - loss: 0.2459 - accuracy: 0.9087 - val_loss: 0.4504 - val_accuracy: 0.8004
Epoch 00003: early stopping


In [129]:
model1.fit(padded_docs, tweets['target'],
          batch_size=76,
          epochs=2,
          verbose=2)

Epoch 1/2
98/98 - 2s - loss: 0.4884 - accuracy: 0.7702
Epoch 2/2
98/98 - 2s - loss: 0.3404 - accuracy: 0.8569


In [130]:
test_result = model1.predict(padded_tests)

In [131]:
test_result

array([[0.62312335],
       [0.81232417],
       [0.7813207 ],
       ...,
       [0.75635624],
       [0.81779766],
       [0.2327313 ]], dtype=float32)

In [132]:
submit = []

for i in test_result:
    if i >= 0.5 :
        submit.append(1)
    else:
        submit.append(0)

submit

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,


In [133]:
tests['target'] = submit

In [134]:
submit_df = tests[['id', 'target']]

In [135]:
submit_df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [136]:
submit_df.to_csv('submit_prueba_38.csv', index=False)